In [1]:
import cv2
import numpy as np

# Load pre-trained MobileNet SSD model
prototxt = "deploy.prototxt"
model = "mobilenet_iter_73000.caffemodel"

# Load the model
net = cv2.dnn.readNetFromCaffe(prototxt, model)

# Open video
cap = cv2.VideoCapture(0)  # Change to 0 for webcam

# Define the virtual counting line
line_y = 300  # Adjust based on your video
count = 0
crossed_ids = set()

# Define class labels (person is class 15 in MobileNet-SSD)
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant",
           "sheep", "sofa", "train", "tvmonitor"]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    h, w = frame.shape[:2]

    # Prepare the frame for detection
    blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    line_x = w // 2  # Place line in the center of the frame

    cv2.line(frame, (line_x, 0), (line_x, h), (0, 255, 255), 2)  


    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.4:  # Confidence threshold
            class_id = int(detections[0, 0, i, 1])
            if CLASSES[class_id] == "person":
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                center_y = (startY + endY) // 2

                # Check if person crosses the line
                if center_y > line_y and i not in crossed_ids:
                    count += 1
                    crossed_ids.add(i)

                # Draw bounding box
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                cv2.putText(frame, f"Person {confidence:.2f}", (startX, startY - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the count
    cv2.putText(frame, f"Count: {count}", (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("People Counter", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
